In [1]:
import numpy as np
from typing import List
from scipy.special import expit
from collections import OrderedDict

In [2]:
class HopfieldModel:
    '''
    Как работает модель.

    Ей подается матрица путей, высчитываются веса и смещение по формулам. Потом ей подается матрица маршрута, причем разные и несколько раз,
    т.к. модель находит локальный минимум, а не глобальный.

    Храним:
    1. Гиперпараметры: A, B, C, D
    2. Веса
    3. Смещение
    4. Слой (название нейронов слоя)
    5. Прошлое значение функции энергии
    6. Название функции активации, которую мы хотим использовать
    7. Сколько раз будем запускать сеть
    8. Финальный путь
    9. Возможно результаты, которые мы захотим сохранить в файл(время, финальный путь)
    10. Стоп параметр
    
    Функции, которые нам нужны:
    1. Функция, которая высчитывает веса и записывает их в переменную
    2. Функция, которая создает рандомный маршрут и возвращает его
    3. Функция активации, которую мы будем использовать
    4. Функция, которая высчитывает значение одного нейрона и возвращает его
    5. Расчет функции энергии
    6. Функция, которая работает с сетью: высчитывает слой в первый раз и итерирует его потом.
    7. Вывол финального пути
    8. Функция для сохранения результатов в файл?
    9. Функция, которая выводит стоимость пути
    

    (неплохо было бы сделать принты, которые бы выводили прогресс)
    Нужно будет еще потом замерять время.
    '''
    
    def __init__(self, fun_activation: str, param: List=[1,1,1,1], iterations: int=5, iterations_limit = 100000) -> None:
        if len(param) != 4:
            raise ValueError('You should put 4 parameters to network') 
        self.A, self.B, self.C, self.D = param
        self.weights = dict()
        self.bias = None
        self.index_list = None
        self.layer = OrderedDict()
        self.energy_fun = None
        if fun_activation not in ['threshold', 'sigmoid']:
            raise ValueError('Function name should be one of it: "threshold", "sigmoid"') 
        self.fun_activation = fun_activation
        self.iterations = iterations
        self.iterations_limit = iterations_limit
        self.final_path = None
        self.total = None
    
    def delta(self, x: int, y: int) -> int:
        return int(x == y)

    # Внедрить хэш функции?
    def calculate_params(self, distant_matrix: List[List]) -> None:
        n = len(distant_matrix)
        self.index_list = [f'{i}{j}' for i in range(n) for j in range(n)]

        # calculate bias
        self.bias = self.C*n
        # calculate weights
        for index_i in self.index_list:
            for index_j in self.index_list:
                if index_i not in self.weights:
                    self.weights[index_i] = dict()
                if index_i == index_j:
                    self.weights[index_i][index_j] = 0
                else:
                    city_1 = int(index_i[0])
                    city_2 = int(index_j[0])
                    order_1 = int(index_i[1])
                    order_2 = int(index_j[1])
                    d = self.delta
                    self.weights[index_i][index_j] = -self.A*d(city_1, city_2)*(1 - d(order_1, order_2)) 
                    self.weights[index_i][index_j] -= self.B*d(order_1, order_2)*(1 - d(city_1, city_2)) 
                    self.weights[index_i][index_j] -= self.C 
                    self.weights[index_i][index_j] -= self.D*distant_matrix[city_1][city_2]*(d(order_2, order_1+1) + d(order_2, order_1-1))
        

    # def create_path(city_number: int) -> List[List]:
    #     return np.random.random_integers(0, 1, (city_number, city_number))

    def fun(self, value: float) -> float:
        if self.fun_activation == 'threshold':
            if value > 0:
                return 1
            else:
                return 0
        elif self.fun_activation == 'sigmoid':
            # return 1/(1 + np.exp(-value))
            return expit(value)
    
    def calculate_neuron(self, current_neuron, previous_layer) -> None:
        self.layer[current_neuron] = np.sum([previous_layer[j]*self.weights[current_neuron][index_j] for j, index_j in enumerate(self.index_list)]) + self.bias
        self.layer[current_neuron] = self.fun(self.layer[current_neuron])

    def calculate_energy_fun(self) -> float:
        e_fun = 0
        for index_i in self.index_list:
            for index_j in self.index_list:
                e_fun -= self.weights[index_i][index_j]*self.layer[index_i]*self.layer[index_j]
            e_fun -= 2*self.bias*self.layer[index_i]
        return e_fun
    

    # Пока что я не могу считать это верно, потому что, чтобы это посчитать мы еще должны знать, какой был прошлый город 
    def calculate_distance(self, distant_matrix: List[List]) -> float:
        total = 0
        # for index in self.index_list:
        #     if int(self.final_path[index]) == 1:
        #         total += distant_matrix[int(index[0])][int(index[1])]
        return total

    def fit(self, distant_matrix: List[List]) -> None:
        self.calculate_params(distant_matrix)
        city_number = len(distant_matrix)
        for i in range(self.iterations):
            path = np.random.randint(2, size=city_number**2)
            for index_i in self.index_list:
                self.calculate_neuron(index_i, path)
            self.energy_fun = self.calculate_energy_fun()
            k = 0
            while True:
                neuron = np.random.choice(self.index_list, 1)[0]
                # print('Past neuron', self.layer[neuron])
                self.calculate_neuron(neuron, list(self.layer.values()))
                new_e_fun = self.calculate_energy_fun()
                if new_e_fun > self.energy_fun:
                    break
                else:
                    self.energy_fun = new_e_fun
                    self.final_path = self.layer
                # print('New neuron', self.layer[neuron])
                # print(f'Step {k}, neuron {neuron}, energy_fun {self.energy_fun}')
                if k == self.iterations_limit:
                    break
                k += 1   

        self.total = self.calculate_distance(distant_matrix)


    def print_result(self) -> None:
        print(self.total)
        for index in self.index_list:
            if int(self.final_path[index]) == 1:
                print(f'City #{index[0]} order in path {index[1]}')

In [3]:
# Чтобы сеть сходилась, матрицы должны быть симметричны
M = 0
ex_1 = np.array([
    [M, 10, 1, 1],
    [10, M, 1, 5],
    [1, 1, M, 10],
    [1, 5, 10, M]
])

ex_2 = np.array([
    [M, 10, 2],
    [10, M, 1],
    [2, 1, M],
])


In [4]:
# [1000000, 1000000, 1000000, 500], 5 - one of the best sets for TSP
model = HopfieldModel('sigmoid', [5000, 5000, 5000, 500], 1)
model.fit(ex_2)
model.print_result()

0
City #0 order in path 0
City #1 order in path 2
City #2 order in path 1


In [5]:
model = HopfieldModel('sigmoid', [5000, 5000, 5000, 500], 1)
model.fit(ex_1)
model.print_result()

0
City #0 order in path 1
City #1 order in path 3
City #2 order in path 0
City #3 order in path 2
